In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
0,Live Updates: COVID-19 news from across the Un...,https://www.cbsnews.com/live-updates/covid-19-...,"Fri, 30 Oct 2020 15:59:00 GMT",2020-10-30 15:59:00+00:00
1,COVID-19 Daily Update 10-30-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Fri, 30 Oct 2020 14:06:40 GMT",2020-10-30 14:06:40+00:00
8,Where Are New COVID-19 Cases In Illinois Comin...,https://www.wbez.org/stories/as-covid-19-surge...,"Fri, 30 Oct 2020 11:00:00 GMT",2020-10-30 11:00:00+00:00
9,Latest on COVID-19 in MN: Daily cases top reco...,https://www.mprnews.org/story/2020/10/30/lates...,"Fri, 30 Oct 2020 10:00:00 GMT",2020-10-30 10:00:00+00:00
2,Hydroxychloroquine in patients with rheumatic ...,https://www.thelancet.com/journals/lanrhe/arti...,"Fri, 30 Oct 2020 00:00:15 GMT",2020-10-30 00:00:15+00:00
6,COVID-19 transmission—up in the air The Lancet,https://www.thelancet.com/journals/lanres/arti...,"Fri, 30 Oct 2020 00:00:15 GMT",2020-10-30 00:00:15+00:00
7,FREE COVID-19 Testing Events Added for Eastern...,https://dhhr.wv.gov/News/2020/Pages/FREE-COVID...,"Thu, 29 Oct 2020 21:19:10 GMT",2020-10-29 21:19:10+00:00
4,Governor Cuomo Announces New Record High Numbe...,https://www.governor.ny.gov/news/governor-cuom...,"Thu, 29 Oct 2020 18:49:08 GMT",2020-10-29 18:49:08+00:00
3,COVID-19 Daily Update 10-29-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Thu, 29 Oct 2020 14:01:32 GMT",2020-10-29 14:01:32+00:00
5,"Coronavirus (COVID-19) Update: October 28, 202...",https://www.fda.gov/news-events/press-announce...,"Wed, 28 Oct 2020 21:51:00 GMT",2020-10-28 21:51:00+00:00
